# Functional Preprocessing

This notebooks preprocesses functional MRI images by executing the following processing steps:

1. Reorient Images to RAS
1. Removal of non-steady state volumes 
1. Motion Correction with SPM
1. Slice-wise Correction with SPM
1. Brain Extraction with SPM and FSL
1. Temporal Filter with Nilearn
1. Two- step coregistration using BBR with FSL, using WM segmentation from SPM
1. Spatial Filter (i.e. smoothing) with Nilearn

Additional, this workflow also performs:
 - ACompCor
 - TCompCor
 - Artifact Detection
 - Computes Friston's 24-paramter model for motion parameters
 
**Note:** This notebook requires that the anatomical preprocessing pipeline was already executed and that it's output can be found in the dataset folder under `dataset/derivatives/fmriflows/preproc_anat`. 

## Data Structure Requirements

The data structure to run this notebook should be according to the BIDS format. Note that the data should be in a session subfolder:

    dataset
    ├── analysis-func_specs.json
    ├── sub-{sub_id}
    │   └── ses-{sess_id}
    │       └── func
    │           ├── sub-{sub_id}_ses-{sess_id}_task-{task_id}_run-{run_id}_bold.nii.gz
    └── task-{task_id}_bold.json

## Execution Specifications

This notebook will extract the relevant processing specifications from the `analysis-func_specs.json` file in the dataset folder. In the current setup, they are as follows:

In [ ]:
import json
from os.path import join as opj

spec_file = opj('/data', 'analysis-func_specs.json')

with open(spec_file) as f:
    specs = json.load(f)

specs

If you'd like to change any of those values manually, overwrite them below:

In [ ]:
# List of subject names
subject_list = specs['subject_list']

# List of session names
session_list = specs['session_list']

# List of run names
run_list = specs['run_list']

# List of task names
task_list = specs['task_list']

# Mode and width of spatial filter, i.e. Low-Pass, fwhm of 6mm = ['LP', 6]
filters_spatial = specs['filters_spatial']

# High and low-pass filter to apply, i.e. Low-Pass of 100Hz = ["None", 100]
filters_temporal = specs['filters_temporal']

# Parameters for Artifact Detection
art_param = specs['artifact_parameters']
norm_threshold = art_param['norm_threshold']
zintensity_threshold = art_param['zintensity_threshold']
use_differences = [b=='True' for b in art_param['use_differences']]

# Requested isometric voxel resolution after coregistration
voxel_res = specs['voxel_res']

# Reference time point (in ms) required for slice wise correction
ref_time = specs['ref_timepoint']

# Number of components to extract with ACompCor and TCompCor
ncomp = specs['n_confound_comp']

# Number of cores to use
n_proc = specs['n_parallel_jobs']

# Cerating the Workflow

To ensure a good overview of the functional preprocessing, the workflow was divided into three subworkflows:

1. The Main Workflow, i.e. doing the actual preprocessing
2. The Confound Workflow, i.e. computing confound variables
3. Visualization Workflow, i.e. visualizating relevant steps for quality control

## Import Modules

In [ ]:
import os
import numpy as np
from os.path import join as opj
from nipype import Workflow, Node, IdentityInterface, Function
from nipype.interfaces.image import Reorient
from nipype.interfaces.spm import SliceTiming, Realign
from nipype.interfaces.fsl import FLIRT, MeanImage, BET, BinaryMaths, ExtractROI
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.algorithms.misc import Gunzip
from nipype.algorithms.rapidart import ArtifactDetect
from nipype.algorithms.confounds import ACompCor, TCompCor, NonSteadyStateDetector, FramewiseDisplacement

# Specify SPM location
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('/opt/spm12-dev/spm12_mcr/spm/spm12')

## Relevant Execution Variables

In [ ]:
# Folder paths and names
exp_dir = '/data/derivatives'
out_dir = 'fmriflows'
work_dir = '/workingdir'

## Create a subworkflow for the Main Workflow

### Implement Nodes

In [ ]:
# Reorient anatomical images to RAS
reorient = Node(Reorient(orientation='RAS'), name='reorient')

In [ ]:
# Detection of Non-Steady State volumes
nonsteady_detection = Node(NonSteadyStateDetector(), name='nonsteady_detection')

In [ ]:
# Store number of non-steady state volumes in text file
def write_to_txt(in_file, n_volumes):
    
    import numpy as np
    out_file = in_file.replace('.nii.gz', '_nss.txt')
    np.savetxt(out_file, [n_volumes], fmt='%d')
    return out_file

write_nss = Node(Function(input_names=['in_file', 'n_volumes'],
                          output_names=['out_file'],
                          function=write_to_txt),
                 name='write_nss')

In [ ]:
# Removal of Non-Steady State volumes
nonsteady_removal = Node(ExtractROI(output_type='NIFTI',
                                    t_size=-1),
                         name='nonsteady_removal')

In [ ]:
# Extract sequence specifications of functional images
def get_parameters(task_id):
    
    import json
    import numpy as np
    from os.path import join as opj
        
    func_desc = opj('/data', 'task-%s_bold.json' % task_id)

    with open(func_desc) as f:
        func_desc = json.load(f)

    # Read out relevant parameters
    TR = func_desc['RepetitionTime']
    slice_order = func_desc['SliceTiming']
    nslices = len(slice_order)
    time_acquisition = float(TR)-(TR/nslices)
    
    return TR, slice_order, nslices, time_acquisition

getParam = Node(Function(input_names=['task_id'],
                         output_names=['TR', 'slice_order',
                                       'nslices', 'time_acquisition'],
                         function=get_parameters),
                name='getParam')

In [ ]:
# Correct for motion
realign = Node(Realign(register_to_mean=True), name='realign')

In [ ]:
# Correct for slice-wise acquisition
slicetime = Node(SliceTiming(ref_slice=ref_time), name='slicetime')

In [ ]:
# Reset TR value after SPM's slice time correction
def add_TR_to_file(in_file, TR):
    
    import nibabel as nb
    
    # Load image
    img = nb.load(in_file)
    
    # Reset TR
    img.header.set_zooms(list(img.header.get_zooms()[:3]) + [TR])
    
    # Save file
    out_file = in_file.replace('.nii', '_TR.nii')
    img.to_filename(out_file)
    del img
    
    return out_file

reset_TR = Node(Function(input_names=['in_file', 'TR'],
                         output_names=['out_file'],
                         function=add_TR_to_file),
                name='reset_TR')

In [ ]:
# Remove skull signal from functional images
bet_func = Node(BET(functional=True,
                    mask=True,
                    output_type='NIFTI_GZ'),
                name='bet_func')

In [ ]:
# Computes mean image before coregistration
bet_mean = Node(MeanImage(dimension='T',
                          output_type='NIFTI_GZ'),
                name='bet_mean')

In [ ]:
# Pre-alignment of functional images to anatomical image
coreg_pre = Node(FLIRT(dof=6,
                       output_type='NIFTI_GZ'),
                 name='coreg_pre')

In [ ]:
# Coregistration of functional images to anatomical image with BBR
# using WM segmentation
coreg_bbr = Node(FLIRT(dof=6,
                       cost='bbr',
                       schedule=opj(os.getenv('FSLDIR'),
                                    'etc/flirtsch/bbr.sch'),
                       output_type='NIFTI_GZ'),
                 name='coreg_bbr')

In [ ]:
# Apply coregistration warp to functional images
applycoreg = Node(FLIRT(interp='spline',
                        apply_isoxfm=voxel_res,
                        datatype='short',
                        output_type='NIFTI_GZ'),
                 name='applycoreg')

In [ ]:
# Crop coregistered files to reduce file size
def crop_img(in_file, reference, padding=3):

    import numpy as np
    import nibabel as nb
    from nilearn.image import coord_transform

    # Load functional and reference image
    func = nb.load(in_file)
    ref = nb.load(reference)

    # Compute the minimal and maximal MNI coordinate of content
    content = np.nonzero(ref.get_fdata())
    corner_min = np.array(content).min(axis=1)
    corner_max = np.array(content).max(axis=1)
    x, y, z = corner_min
    min_coord = coord_transform(x, y, z, ref.affine)
    x, y, z = corner_max
    max_coord = coord_transform(x, y, z, ref.affine)

    # Transform min and max coordinates back into functional space.
    # Padding value can be used to extend the cut area.
    inverse = np.linalg.inv(func.affine)
    cut_min = np.dot(inverse, np.hstack((min_coord, 1))
                     )[:3].round().astype('int') - padding
    cut_max = np.dot(inverse, np.hstack((max_coord, 1))
                    )[:3].round().astype('int') + padding

    # Make sure that cuts are not outside of volume
    cut_min[cut_min < 0] = 0
    outside_box = 100+cut_max > func.shape[:3]
    cut_max[outside_box] = np.array(func.shape[:3])[outside_box]

    # Reduce FOV of functional image accordingly
    func = func.slicer[cut_min[0]: cut_max[0],
                       cut_min[1]: cut_max[1],
                       cut_min[2]: cut_max[2],
                       :]
    out_file = in_file.replace('.nii', '_crop.nii')
    func.to_filename(out_file)
    del func

    return out_file

cropper = Node(Function(input_names=['in_file', 'reference'],
                        output_names=['out_file'],
                        function=crop_img),
               name='cropper')

In [ ]:
# Apply Temporal Filter
def apply_temporal_filter(in_file, TR, tFilter):
    
    import nibabel as nb
    from nilearn.image import clean_img, mean_img, math_img
    
    # Transform cutoff values into HZ
    low_pass, high_pass = tFilter
    postfix = 'tfilt_%s.%s' % (low_pass, high_pass)
    low_pass = 1. / low_pass if low_pass != 'None' else None
    high_pass = 1. / high_pass if high_pass != 'None' else None
    
    out_file = in_file.replace('.nii', '_%s.nii' % postfix)
    
    # Apply temporal filter and store it in new file
    img = clean_img(in_file, detrend=False, standardize=False, t_r=TR,
                    ensure_finite=True, low_pass=low_pass, high_pass=high_pass)
    affine = img.affine
    header = img.header

    # Add mean if image was high pass filtered
    if high_pass:
        img = math_img("img1 + img2[...,None]", img1=img, img2=mean_img(in_file))

    # Save temporal filtered image
    nb.Nifti1Image(img.get_fdata(), affine, header).to_filename(out_file)
    del img

    return out_file

temporal_filter = Node(Function(input_names=['in_file', 'TR', 'tFilter'],
                        output_names=['out_file'],
                        function=apply_temporal_filter),
               name='temp_filter')
temporal_filter.iterables = ('tFilter', filters_temporal)

In [ ]:
# Applies gaussian spatial filter as in Sengupta, Pollmann & Hanke, 2018
def gaussian_spatial_filter(in_file, sFilter, bandwidth=1):

    import nibabel as nb
    from nilearn.image import smooth_img

    ftype, fwhm = sFilter

    if ftype == 'LP':
        img = smooth_img(in_file, fwhm=fwhm)
        
    if ftype == 'HP':
        img = nb.load(in_file)
        HPF_bold = img.get_data() - smooth_img(in_file, fwhm=fwhm).get_data()
        img = nb.Nifti1Image(HPF_bold, img.get_affine())
        
    elif ftype == 'BP':
        LPF_bold_1 = smooth_img(in_file, fwhm=fwhm)
        LPF_bold_2 = smooth_img(in_file, fwhm=fwhm - bandwidth)
        BPF_bold = LPF_bold_2.get_fdata() - LPF_bold_1.get_fdata()
        img = nb.Nifti1Image(BPF_bold, LPF_bold_1.affine, LPF_bold_1.header)
        
    # Save and return output file
    out_file = in_file.replace('.nii', '_%s_%smm.nii' % (ftype, fwhm))
    img.to_filename(out_file)
    del img

    return out_file

# Spatial Band-Pass Filter
spatial_filter = Node(Function(input_names=['in_file', 'sFilter'],
                        output_names=['out_file'],
                        function=gaussian_spatial_filter),
               name='spatial_filter')
spatial_filter.iterables = ('sFilter', filters_spatial)

In [ ]:
# Computes mean image
meanimg = Node(MeanImage(dimension='T',
                         output_type='NIFTI_GZ'),
               name='meanimg')

### Create Main Workflow

**Note:** Slice time correction is applied after motion correction, as recommended by Power et al. (2017): http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0182939

In [ ]:
# Create main preprocessing workflow
mainflow = Workflow(name='mainflow')

In [ ]:
# Add nodes to workflow and connect them
mainflow.connect([(reorient, nonsteady_detection, [('out_file', 'in_file')]),
                  (reorient, nonsteady_removal, [('out_file', 'in_file')]),
                  (reorient, write_nss, [('out_file', 'in_file')]),
                  (nonsteady_detection, write_nss, [('n_volumes_to_discard', 'n_volumes')]),
                  (nonsteady_detection, nonsteady_removal, [('n_volumes_to_discard',
                                                             't_min')]),
                  (nonsteady_removal, realign, [('roi_file', 'in_files')]),
                  (realign, slicetime, [('realigned_files', 'in_files')]),
                  (getParam, slicetime, [('TR', 'time_repetition'),
                                         ('slice_order', 'slice_order'),
                                         ('nslices', 'num_slices'),
                                         ('time_acquisition', 'time_acquisition'),
                                         ]),
                  (slicetime, reset_TR, [('timecorrected_files', 'in_file')]),
                  (getParam, reset_TR, [('TR', 'TR')]),
                  (reset_TR, bet_func, [('out_file', 'in_file')]),
                  (bet_func, bet_mean, [('out_file', 'in_file')]),

                  # Coregistration
                  (coreg_pre, coreg_bbr, [('out_matrix_file', 'in_matrix_file')]),
                  (coreg_bbr, applycoreg, [('out_matrix_file', 'in_matrix_file')]),
                  (bet_mean, coreg_pre, [('out_file', 'in_file')]),
                  (bet_mean, coreg_bbr, [('out_file', 'in_file')]),
                  (bet_func, applycoreg, [('out_file', 'in_file')]),
                  (applycoreg, cropper, [('out_file', 'in_file')]),
                  
                  # Apply Temporal and Spatial Filter
                  (getParam, temporal_filter, [('TR', 'TR')]),
                  (cropper, temporal_filter, [('out_file', 'in_file')]),
                  (temporal_filter, spatial_filter, [('out_file', 'in_file')]),
                  (cropper, meanimg, [('out_file', 'in_file')]),
                  ])

## Create a subworkflow for the Confound Workflow

### Implement Nodes

In [ ]:
# Run ACompCor (based on Behzadi et al., 2007)
aCompCor = Node(ACompCor(num_components=ncomp,
                         pre_filter=False,
                         save_pre_filter=False,
                         merge_method='union',
                         components_file='compcorA.txt'),
                name='aCompCor')

In [ ]:
# Create binary mask for ACompCor (based on Behzadi et al., 2007)
def get_csf_wm_mask(wm, csf, in_file):
    
    from nibabel import Nifti1Image
    from nilearn.image import threshold_img, resample_to_img
    from scipy.ndimage.morphology import binary_erosion, binary_closing

    # Create eroded WM binary mask
    thr_wm = threshold_img(wm, 0.99)
    res_wm = resample_to_img(thr_wm, in_file)
    bin_wm = threshold_img(res_wm, 0.5)
    mask_wm = binary_erosion(bin_wm.get_fdata(), iterations=2).astype('int8')

    # Create eroded CSF binary mask (differs from Behzadi et al., 2007)
    thr_csf = threshold_img(csf, 0.99)
    res_csf = resample_to_img(thr_csf, in_file)
    bin_csf = threshold_img(res_csf, 0.5)
    close_csf = binary_closing(bin_csf.get_fdata(), iterations=1)
    mask_csf = binary_erosion(close_csf, iterations=1).astype('int8')
    
    # Combine WM and CSF binary masks into one
    binary_mask = ((mask_wm + mask_csf) > 0).astype('int8')
    out_file = in_file.replace('.nii', '_maskA.nii')
    Nifti1Image(binary_mask, res_wm.affine).to_filename(out_file)

    return out_file

acomp_masks = Node(Function(input_names=['wm', 'csf', 'in_file'],
                            output_names=['out_file'],
                            function=get_csf_wm_mask),
                   name='acomp_masks')

In [ ]:
# Run TCompCor (based on Behzadi et al., 2007)
tCompCor = Node(TCompCor(num_components=ncomp,
                         percentile_threshold=0.02,
                         pre_filter=False,
                         save_pre_filter=False,
                         components_file='compcorT.txt'),
                name='tCompCor')

In [ ]:
# Create binary mask for TCompCor approach (based on Behzadi et al., 2007)
def get_brainmask(in_file):
    
    from nibabel import Nifti1Image
    from nilearn.image import mean_img
    from scipy.ndimage.morphology import binary_erosion
    
    img = mean_img(in_file)
    erod_img = binary_erosion(img.get_fdata()>0, iterations=1).astype('int8')
    
    out_file = in_file.replace('.nii', '_maskT.nii')
    Nifti1Image(erod_img, img.affine).to_filename(out_file)
    
    return out_file

tcomp_brainmask = Node(Function(input_names=['in_file'],
                          output_names=['out_file'],
                          function=get_brainmask),
                 name='tcomp_brainmask')

In [ ]:
# Compute FramewiseDisplacement
FD = Node(FramewiseDisplacement(parameter_source='SPM'),
          name='FD')

In [ ]:
# Detects intensity and motion artifacts and labels them as outliers
# Note that a z-threshold of 2.58 corresponds to 99% of normal distributed vales
art = Node(ArtifactDetect(norm_threshold=norm_threshold,
                          zintensity_threshold=zintensity_threshold,
                          mask_type='file',
                          parameter_source='SPM',
                          use_differences=use_differences,
                          plot_type='svg'),
           name='art')

In [ ]:
# Computes Friston 24-parameter model (Friston et al., 1996)
def compute_friston24(in_file):
    
    import numpy as np
    
    # Load raw motion parameters
    mp_raw = np.loadtxt(in_file)
    
    # Get motion paremter one time point before
    mp_minus1 = np.vstack((mp_raw[1:], [0] * 6))
    
    # Combine the two
    mp_combine = np.hstack((mp_raw, mp_minus1))

    # Add the square of those parameters to allow correction of nonlinear effects
    mp_friston = np.hstack((mp_combine, mp_combine**2))

    # Save friston 24-parameter model in new txt file
    out_file = in_file.replace('.txt', 'friston24.txt')
    np.savetxt(out_file, mp_friston,
               fmt='%.8f', delimiter=' ', newline='\n')
    
    return out_file

friston24 = Node(Function(input_names=['in_file'],
                          output_names=['out_file'],
                          function=compute_friston24),
                 name='friston24')

In [ ]:
# Combine confound parameters into one TSV file
def consolidate(FD, par_rp, par_friston, compA, compT):
    
    import numpy as np
    
    conf_FD = np.array(list(np.loadtxt(FD, skiprows=1)) + [0])
    conf_rp = np.loadtxt(par_rp)
    conf_friston = np.loadtxt(par_friston)
    conf_compA = np.loadtxt(compA, skiprows=1)
    conf_compT = np.loadtxt(compT, skiprows=1)

    # Aggregate confounds
    confounds = np.hstack((conf_FD[..., None],
                           conf_rp,
                           conf_friston,
                           conf_compA,
                           conf_compT))

    # Create header
    header = ['FD']
    header += ['RP%02d' % (d + 1) for d in range(conf_rp.shape[1])]
    header += ['Friston%02d' % (d + 1) for d in range(conf_friston.shape[1])]
    header += ['CompA%02d' % (d + 1) for d in range(conf_compA.shape[1])]
    header += ['CompT%02d' % (d + 1) for d in range(conf_compT.shape[1])]

    # Write to file
    out_file = par_rp.replace('rp', 'confounds')
    out_file = out_file.replace('.txt', '.tsv')
    with open(out_file, 'w') as f:
        f.write('\t'.join(header) + '\n')
        for row in confounds:
            f.write('\t'.join([str(r) for r in row]) + '\n')
    
    return out_file

combine_confounds = Node(Function(input_names=['FD', 'par_rp', 'par_friston',
                                               'compA', 'compT'],
                                  output_names=['out_file'],
                                  function=consolidate),
                         name='combine_confounds')

In [ ]:
# Compute Confound Correlation Maps
def compute_correlation_map(in_file, confounds):

    import numpy as np
    import nibabel as nb
    from scipy.stats import zscore
    
    # Load image
    img = nb.load(in_file)

    # zscore functional data
    data = zscore(img.get_fdata(), axis=-1)

    # zscore confound parameters
    par = zscore(np.loadtxt(confounds, skiprows=1), axis=-1)

    # Compute correlation map per component
    corr_map = np.nan_to_num(np.dot(data, par))

    # Save and return output file
    img = nb.Nifti1Image(corr_map, img.affine, img.header)
    out_file = in_file.replace('.nii', '_confounds_map.nii')
    img.to_filename(out_file)
    del img, data

    return out_file

comp_corr_map = Node(Function(input_names=['in_file', 'confounds'],
                              output_names=['out_file'],
                              function=compute_correlation_map),
                     name='comp_corr_map')

### Create Confound Workflow

In [ ]:
# Create confound extraction workflow
confflow = Workflow(name='confflow')

In [ ]:
# Add nodes to workflow and connect them
confflow.connect([(acomp_masks, aCompCor, [('out_file', 'mask_files')]),
                  (tcomp_brainmask, tCompCor, [('out_file', 'mask_files')]),

                  # Consolidate confounds
                  (FD, combine_confounds, [('out_file', 'FD')]),
                  (aCompCor, combine_confounds, [('components_file', 'compA')]),
                  (tCompCor, combine_confounds, [('components_file', 'compT')]),
                  (friston24, combine_confounds, [('out_file', 'par_friston')]),
                  
                  # Compute functional correlation map
                  (combine_confounds, comp_corr_map, [('out_file', 'confounds')]),
                  ])
confflow.add_nodes([art])

## Specify Input & Output Stream

In [ ]:
# Iterate over subject, session, task and run id
infosource = Node(IdentityInterface(fields=['subject_id', 'session_id',
                                            'task_id', 'run_id']),
                  name='infosource')
infosource.iterables = [('subject_id', subject_list),
                        ('session_id', session_list),
                        ('task_id', task_list),
                        ('run_id', run_list)]

In [ ]:
# Specify input file location
templates = {'anat':  opj('fmriflows', 'preproc_anat', 'sub-{subject_id}',
                          'ses-{session_id}_brain.nii.gz'),
             'wm':  opj('fmriflows', 'preproc_anat', 'sub-{subject_id}',
                        'ses-{session_id}_seg_wm.nii'),
             'csf':  opj('fmriflows', 'preproc_anat', 'sub-{subject_id}',
                         'ses-{session_id}_seg_csf.nii'),
             'func':  opj('/data', 'sub-{subject_id}', 'ses-{session_id}', 'func',
                          'sub-{subject_id}_ses-{session_id}_task-{task_id}_run-{run_id}_bold.nii.gz')}

sf = Node(SelectFiles(templates,
                      base_directory=exp_dir,
                      sort_filelist=True),
          name='selectfiles')

In [ ]:
# Save relevant outputs in a datasink
datasink = Node(DataSink(base_directory=exp_dir,
                         container=out_dir),
                name='datasink')

In [ ]:
# Apply the following naming substitutions for the datasink
substitutions = [(
    '_run_id_%s_session_id_%s_subject_id_%s_task_id_%s/' % (run, sess, sub, task),
    'sub-%s/task-%s_ses-%s_run-%s_' % (sub, task, sess, run))
    for sub in subject_list
    for sess in session_list
    for run in run_list
    for task in task_list]
substitutions += [
    ('sub-%s_ses-%s_task-%s_run-%s_bold' % (sub, sess, task, run), '')
    for sub in subject_list
    for sess in session_list
    for run in run_list
    for task in task_list]
substitutions += [('_tFilter_%s.%s/' % (t[0], t[1]), '')
                  for t in filters_temporal]
substitutions += [('_sFilter_%s.%s/' % (s[0], s[1]), '')
                  for s in filters_spatial]
substitutions += [('%s_%smm' % (s[0], s[1]),
                   'sFilter_%s_%smm' % (s[0], s[1]))
                  for s in filters_spatial]
substitutions += [('ar_', ''),
                  ('ras_', ''),
                  ('roi_', ''),
                  ('TR_', ''),
                  ('brain_', ''),
                  ('flirt_', ''),
                  ('crop_', ''),
                  ('tfilt', 'tFilter'),
                  ('mask_000', 'maskT'),
                  ('art.r_', ''),
                  ('_roi', '_'),
                  ('__', '_'),
                  ('_.', '.'),
                  ('.r.', '.'),
                  ]
datasink.inputs.substitutions = substitutions

## Implement Functional Preprocessing Workflow

In [ ]:
# Create functional preprocessing workflow
preproc_func = Workflow(name='preproc_func')
preproc_func.base_dir = work_dir

# Connect input nodes to each other
preproc_func.connect([(infosource, sf, [('subject_id', 'subject_id'),
                                        ('session_id', 'session_id'),
                                        ('task_id', 'task_id'),
                                        ('run_id', 'run_id')])])

In [ ]:
# Add input and output nodes and connect them to the main workflow
preproc_func.connect([(infosource, mainflow, [('task_id', 'getParam.task_id')]),
                      (sf, mainflow, [('func', 'reorient.in_file'),
                                      ('anat', 'coreg_pre.reference'),
                                      ('anat', 'coreg_bbr.reference'),
                                      ('wm', 'coreg_bbr.wm_seg'),
                                      ('anat', 'applycoreg.reference'),
                                      ('anat', 'cropper.reference')]),
                      
                      (mainflow, datasink, [
                          ('spatial_filter.out_file', 'preproc_func.@func'),
                          ('meanimg.out_file', 'preproc_func.@mean'),
                          ('write_nss.out_file', 'preproc_func.@nss')]),
                     ])

In [ ]:
# Add input and output nodes and connect them to the confound workflow
preproc_func.connect([(sf, confflow, [('wm', 'acomp_masks.wm'),
                                      ('csf', 'acomp_masks.csf')]),

                      (confflow, datasink, [
                          ('art.outlier_files', 'preproc_func.@outlier_files'),
                          ('art.plot_files', 'preproc_func.@outlier_plot'),
                          ('tCompCor.high_variance_masks', 'preproc_func.@maskT'),
                          ('acomp_masks.out_file', 'preproc_func.@maskA'),
                          ('combine_confounds.out_file', 'preproc_func.@confound_tsv'),
                          ('comp_corr_map.out_file', 'preproc_func.@confound_map')
                      ]),
                       ])

In [ ]:
# Connect main workflow with confound workflow
preproc_func.connect([(mainflow, confflow, [
                          ('getParam.TR', 'aCompCor.repetition_time'),
                          ('cropper.out_file', 'aCompCor.realigned_file'),
                          ('cropper.out_file', 'acomp_masks.in_file'),
                          ('getParam.TR', 'tCompCor.repetition_time'),
                          ('cropper.out_file', 'tCompCor.realigned_file'),
                          ('cropper.out_file', 'tcomp_brainmask.in_file'),
                          ('cropper.out_file', 'comp_corr_map.in_file'),
                          ('realign.realigned_files', 'art.realigned_files'),
                          ('realign.realignment_parameters', 'art.realignment_parameters'),
                          ('realign.realignment_parameters', 'combine_confounds.par_rp'),
                          ('realign.realignment_parameters', 'friston24.in_file'),
                          ('bet_func.mask_file', 'art.mask_file'),
                          ('realign.realignment_parameters', 'FD.in_file'),
                          ('getParam.TR', 'FD.series_tr'),
                          ])
                     ])

## Visualize Workflow

In [ ]:
# Create preproc_func output graph
preproc_func.write_graph(graph2use='colored', format='svg', simple_form=True)

# Visualize the graph
from IPython.display import SVG
SVG(filename=opj(preproc_func.base_dir, 'preproc_func', 'graph.svg'))

# Run Workflow

In [ ]:
# Run the workflow in parallel mode
preproc_func.run(plugin='MultiProc', plugin_args={'n_procs' : n_proc})

In [ ]:
# Save workflow graph visualizations in datasink
preproc_func.write_graph(graph2use='flat', format='svg', simple_form=True)
preproc_func.write_graph(graph2use='colored', format='svg', simple_form=True)

from shutil import copyfile
copyfile(opj(preproc_func.base_dir, 'preproc_func', 'graph.svg'),
         opj(exp_dir, out_dir, 'preproc_func', 'graph.svg'))
copyfile(opj(preproc_func.base_dir, 'preproc_func', 'graph_detailed.svg'),
         opj(exp_dir, out_dir, 'preproc_func', 'graph_detailed.svg'));

# WIP: To show possible outputs

In [ ]:
sub = subject_list[0]
sess = session_list[0]
run = run_list[0]
task = task_list[0]

In [ ]:
# If needed, create title for output figures
title_txt = 'Sub: %s - Task: %s - Sess: %s - Run: %s' % (sub, task, sess, run)
title_txt

In [ ]:
import numpy as np
from nilearn.image import coord_transform

# Support Function to get optimal cut for visualization
def get_cut_ids(img, axis=0):

    # Compute voxel id to cut
    idx = np.sort(img.get_fdata().nonzero()[axis])
    vox_id = np.linspace(idx.min(), idx.max(), num=12, endpoint=True).astype('int')
    vox_id = vox_id[2:-2]

    # Translate voxel id to image space
    if axis == 0:
        cut_ids = [int(coord_transform(r, 0, 0, img.affine)[0]) for r in vox_id]
    elif axis == 1:
        cut_ids = [int(coord_transform(0, r, 0, img.affine)[1]) for r in vox_id]
    elif axis == 2:
        cut_ids = [int(coord_transform(0, 0, r, img.affine)[2]) for r in vox_id]
    return cut_ids

## Overlay Mean functional (after co-registration) and CompCor masks on anatomy

This plot should help to investigate the co-registration (grey outlienes of functional mean) and shows which voxels are included in the temporal (yellow) and anatomical (blue) CompCor mask.

In [ ]:
anat = '/data/derivatives/fmriflows/preproc_anat/sub-%s/ses-%s_T1w_corrected.nii.gz' % (sub, sess)
mean = '/data/derivatives/fmriflows/preproc_func/sub-%s/task-%s_ses-%s_run-%s_mean.nii.gz' % (sub, task, sess, run)
maskA = '/data/derivatives/fmriflows/preproc_func/sub-%s/task-%s_ses-%s_run-%s_maskA.nii.gz' % (sub, task, sess, run)
maskT = '/data/derivatives/fmriflows/preproc_func/sub-%s/task-%s_ses-%s_run-%s_maskT.nii.gz' % (sub, task, sess, run)

In [ ]:
import nibabel as nb
from matplotlib.pyplot import figure
from nilearn.plotting import plot_anat

# Visualize preprocessed functional mean on subject anatomy
def plot_mean(anat, mean, maksA, maskT, title):
    fig = figure(figsize=(16, 8))
    
    for i, e in enumerate(['x', 'y', 'z']):
        ax = fig.add_subplot(3, 1, i + 1)
        
        display = plot_anat(anat, title=title_txt + ' - %s-axis' % e, colorbar=False,
                            display_mode=e, cut_coords=get_cut_ids(nb.load(mean), i),
                            annotate=False, axes=ax)
        display.add_edges(mean, color='lightgrey')
        display.add_contours(maskA, filled=True, cmap='cool_r',
                             resampling_interpolation='nearest')
        display.add_contours(maskT, filled=True, cmap='Wistia_r',
                             resampling_interpolation='nearest')
    out_file = mean.replace('_mean.nii.gz', '_overlays.svg')
    fig.savefig(out_file, bbox_inches='tight', facecolor='black', frameon=True,
                dpi=300, transparent=True)

plot_mean(anat, mean, maskA, maskT, title_txt)

## Plot artifact detection output

In [ ]:
from IPython.display import SVG
art = '/data/derivatives/fmriflows/preproc_func/sub-%s/task-%s_ses-%s_run-%s_plot.svg' % (sub, task, sess, run)
SVG(filename=art)

## Plot carpet plot

From fmriprep: "Summary statistics are plotted, which may reveal trends or artifacts in the BOLD data. Global signals calculated within the whole-brain (GS), within the white-matter (WM) and within cerebro-spinal fluid (CSF) show the mean BOLD signal in their corresponding masks. DVARS and FD show the standardized DVARS and framewise-displacement measures for each time point.<br />A carpet plot shows the time series for all voxels within the brain mask. Voxels are grouped into cortical (blue), and subcortical (orange) gray matter, cerebellum (green) and white matter and CSF (red), indicated by the color map on the left-hand side."

I wanted to look into this, but a quick try didn't really work. I'm not sure if we can create such a carpet plot with GM, WM and CSF segments or if we really need an atlas. If so, than we can use the script `/templates/parcellate_HarvardOxford.py` to create HarvardOxford atlas ROIs with a desired voxel resolution.

## Plot confounds

In [ ]:
confounds = '/data/derivatives/fmriflows/preproc_func/sub-%s/task-%s_ses-%s_run-%s_confounds.tsv' % (sub, task, sess, run)

In [ ]:
# Visualize confound parameters
def plot_confound_parameters(confounds, title):

    import numpy as np
    import seaborn as sns
    sns.set(context='notebook', style='darkgrid')
    import matplotlib.pyplot as plt

    # Load confound signal and header
    signal = np.loadtxt(confounds, skiprows=1)
    with open(confounds, 'r') as f:
        header = f.readlines()[0][:-1].split('\t')

    # Plot and save Framewise Displacement
    fig = figure(figsize=(12, 2))
    plt.title('FD')
    plt.plot(signal[:, 0])
    plt.ylabel(header[0])
    plt.xlabel('Time [in #TR]')
    plt.tight_layout()
    out_file = confounds.replace('.tsv','_FD.svg')
    fig.savefig(out_file)
    fig.clf()
        
    # Specify plotting groups
    groups = {'Motion': [1, 6],
              'Friston24': [7, 24],
              'CompCorA': [31, 6],
              'CompCorT': [37, 6]}
    
    for g in groups:
        
        cut = groups[g]
        conf = signal[:, cut[0]:cut[0] + cut[1]]
        head = header[cut[0]:cut[0] + cut[1]]
        nConf = conf.shape[-1]
    
        fig, axes = plt.subplots(nConf, 1, figsize=(12, nConf * 2))
        axes[0].set_title(g)
        for i in range(nConf):
            axes[i].plot(conf[:, i])
            axes[i].set_ylabel(head[i])
        axes[i].set_xlabel('Time [in #TR]')
        plt.tight_layout()

        # Save output
        out_file = confounds.replace('.tsv','_%s.svg' % g)
        fig.savefig(out_file)
        fig.clf()
    
plot_confound_parameters(confounds, title_txt)

## Plot confound correlation maps

Here I take the correlation maps from the motion, friston24, compcorA and compcorT confounds and compute the maximum correlation per voxel and plot it on the anatomy.

In [ ]:
anat = '/data/derivatives/fmriflows/preproc_anat/sub-%s/ses-%s_T1w_corrected.nii.gz' % (sub, sess)
corr_map = '/data/derivatives/fmriflows/preproc_func/sub-%s/task-%s_ses-%s_run-%s_confounds_map.nii.gz' % (sub, task, sess, run)

In [ ]:
# Visualize confound correlation maps
def plot_confounds(anat, corr_map, title):

    import nibabel as nb
    from matplotlib.pyplot import figure
    from nilearn.plotting import plot_stat_map
    
    # Load correlation map
    corr = nb.load(corr_map)
    
    # Specify plotting groups
    groups = {'Motion': [1, 6],
              'Friston24': [7, 24],
              'CompCorA': [31, 6],
              'CompCorT': [37, 6]}
   
    for g in groups:
        
        cut = groups[g]
        conf = corr.slicer[..., cut[0]:cut[0] + cut[1]]
        max_corr = np.max(np.abs(conf.get_fdata()), axis=-1)
        max_conf = nb.Nifti1Image(max_corr, conf.affine, conf.header)

        fig = figure(figsize=(16, 8))
        for i, e in enumerate(['x', 'y', 'z']):
            ax = fig.add_subplot(3, 1, i + 1)

            thr = max_conf.get_fdata().max() * 0.50
            
            plot_stat_map(max_conf, title=title_txt + ' - %s' % g, colorbar=False,
                      threshold=thr, bg_img=anat, display_mode=e,
                      resampling_interpolation='nearest',
                      cut_coords=get_cut_ids(max_conf, i),
                      cmap='magma', annotate=False, axes=ax)
            
        out_file = corr_map.replace('.nii.gz', '_%s.svg' % g)
        fig.savefig(out_file, bbox_inches='tight', facecolor='black', frameon=True,
                    dpi=300, transparent=True)

plot_confounds(anat, corr_map, title_txt)